# MICS Download

This notebook will call the MICS_module and use the functions to download and sort new MICS data.

### Instructions 
1. When a new MICS survey is avaialable, go to mics survey site (https://mics.unicef.org/surveys). Click the upload button and select CSV next to the grayed-out "DOWNLOAD MICS DATASETS" button to get the most up to date metadata.

2. Select the correct survey round, the appropriate region and select MICS as the datatype. Click "DOWNLOAD MICS DATASETS".

    **NOTE** The MICS_module is made to sort bulk mics datasets. If you download only one dataset from a country this function will not work and it would be best to manually sort that into the appropriate folder

3. Import the MICS_module into you notebook.

4. Use MICS_module to process the downloaded metadata csv from step 1:

    ```{python}
    metadata = MICS_module.process_mics_metadata('path_to_metadata', 'MICS/ISO3_country_codes.csv')
    ``` 

5.  Check for missing values in metadata and fill in as needed. There shouldn't be any missing values right now, but as country names change, there might be.
The MICS metadata is organized as follows:  
    - round: The MICS round (e.g., "MICS6").
    - round_num: The numeric representation of the MICS round (e.g., 6).
    - country_x: The original country name from the MICS metadata.
    - country_code: The ISO3 country code.
    - year: The year of the survey.
    - save_name: The standardized country name used for saving files.
    - standardized: The standardized country name for merging and sorting.

6. Use MICS_module to sort the zipped datasets downloaded in step 2:  

    ```{python} 
    MICS_module.extract_and_save_zipped_files('file/extraction/log/path.txt', 'zip/file/path/', mics_metadata, survey_round_number) 
    ```

In [ ]:
#!pip install pycountry
#!pip install unidecode
#!pip install rapidfuzz

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.5 MB/s eta 0:00:00:00:01


In [19]:
#import mics module
import MICS_module

In [16]:
#process mics metadata
metadata = MICS_module.process_mics_metadata('MICS/mics_surveys_catalogue.csv', 'MICS/ISO3_country_codes.csv')

In [22]:
#view metadata
metadata.head()

,round,round_num,country_x,country_code,year,save_name,standardized
0,MICS7,7,Viet Nam,VNM,2027,Vietnam,Viet Nam
1,MICS7,7,Armenia,ARM,2026,Armenia,Armenia
2,MICS7,7,Tuvalu,TUV,2026,Tuvalu,Tuvalu
3,MICS7,7,Fiji,FJI,2026,Fiji,Fiji
4,MICS7,7,Kiribati,KIR,2026,Kiribati,Kiribati


In [23]:
#extract and sort data
MICS_module.extract_and_save_zipped_files('MICS/MICS_error_logs/test_log.txt', 'MICS/MICS_zip/MICS_Dataseets test.zip', metadata, 6)

UnboundLocalError: cannot access local variable 'temp_dir' where it is not associated with a value